In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
import numpy as np
import random as rd
from tqdm import tqdm

In [3]:
action_map = {0:'left',
          1:'down',
          2:'right',
          3:'up'}

# Maze Environment

I have created my own maze environment. The role of this environment is to simulate the action given to the environment and display it.

In [4]:
class Env:
    def __init__(self):
        self.env = np.array([[0,0,0,0],
                            [0,-1,0,-1],
                            [0,0,0,-1],
                            [-1,0,0,2]])

        self.win_reward = 1
        self.loss_reward = -1

        self.current_state = (0,0)
        self.shape = self.env.shape
        self.done = False
        self.reward = 0

    def terminate_state(self):
        row,col = self.current_state
        if self.env[row][col] == 2:
            self.done = True
            self.reward = self.win_reward

        elif self.env[row][col] == -1:
            self.done = True
            self.reward = self.loss_reward

        else:
            self.done = False
            self.reward = 0

    def step(self,action):
        row,col = self.current_state
        if action == 2:
            if col+1 < self.shape[1]:
                self.current_state = (row,col+1)
        if action == 0:
            if col-1 >= 0:
                self.current_state = (row,col-1)
        if action == 1:
            if row-1 >= 0:
                self.current_state = (row-1,col)
        if action == 3:
            if row+1 < self.shape[0]:
                self.current_state = (row+1,col)

        self.terminate_state()
        return self.current_state , self.done , self.reward


    def render(self):
        row,col = self.env.shape
        env_rend = ''
        for i in range(row):
            for j in range(col):
                state = self.env[i][j]
                if self.current_state == (i,j):
                    env_rend += 'S'
                elif state == 0:
                    env_rend += 'F'
                elif state == -1:
                    env_rend += 'H'
                else:
                    env_rend += 'G'
            env_rend += '\n'
        print(env_rend)

In [5]:
env = Env()
obs,done,reward = env.step(2)
print(f'observation : {obs} done : {done} reward : {reward}')

# we can print the environment if we want to look at it
# with a current state S
env.render()

observation : (0, 1) done : False reward : 0
FSFF
FHFH
FFFH
HFFG



In [6]:
obs,done,reward = env.step(3)
print(f'observation : {obs} done : {done} reward : {reward}')

# we can print the environment if we want to look at it
# with a current state S
env.render()

observation : (1, 1) done : True reward : -1
FFFF
FSFH
FFFH
HFFG



In [7]:
env = Env()
env.render()

SFFF
FHFH
FFFH
HFFG



I have labeled each 1-D array to 2D array indexs

In [8]:
numcod = {
    0: (0, 0),    1: (0, 1),    2: (0, 2),    3: (0, 3),    4: (1, 0),
    5: (1, 1),   6: (1, 2),    7: (1, 3),    8: (2, 0),    9: (2, 1),
    10: (2, 2),    11: (2, 3),    12: (3, 0),    13: (3, 1),    14: (3, 2),
    15: (3, 3),
}

# Reward and Loss

The function below sets the reward and loss value for any transition from one state to another:

In [9]:
def setlossRmatrix(loss,rmatrix):
    for i in loss:
        rmatrix[i[0]][i[1]] = -10
    return rmatrix

def setwinRmatrix(win,rmatrix):
    for i in win:
        rmatrix[i[0]][i[1]] = 10
    return rmatrix

# Reward and Q matrix
create a 16x16 Reward and Q matrix  , the loss and win contain the index transition 

In [11]:
qmatrix = np.zeros((16,16))
rmatrix = np.zeros((16,16))

loss = [(1,5),(4,5),(6,5),(9,5),(6,7),(3,7),(11,7),(8,12),(13,12)]
win = [(14,15),(11,15)]

rmatrix = setlossRmatrix(loss,rmatrix)
rmatrix = setwinRmatrix(win,rmatrix)

# Reward Matrix value

In [12]:
rmatrix

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., -10.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0., -10.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., -10.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0., -10.,   0., -10.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0., -10.,   

# Possible Action

This function contains a possible action at any given index in the maze. This value changes when the maze is altered.

In [13]:
posact = {
    0:[2,3],1:[0,2,3],2:[0,2,3],3:[0,3],
    4:[1,2,3],5:[0,1,2,3],6:[0,1,2,3],7:[0,1,3],
    8:[1,2,3],9:[0,1,2,3],10:[0,1,2,3],11:[0,1,3],
    12:[1,2],13:[0,1,2],14:[0,1,2],15:[0,1]
}

# 

In [14]:
y = 0.9
done = False
count = 0
for i in tqdm(range(1000)):
    state1 = 0
    done = False
    env = Env()
    while done != True:
        ran = rd.random()
        action = rd.choice(posact[state1])        
        obs,done,reward = env.step(action)
        state2 = (obs[0]*4)+obs[1]
        qmatrix[state1][state2] = rmatrix[state1][state2] + y * max([qmatrix[state2][a] for a in list(range(16))])  
        state1 = state2        

100%|██████████| 10000/10000 [00:00<00:00, 21474.64it/s]


In [15]:
qmatrix

array([[  0.     ,   5.9049 ,   0.     ,   0.     ,   5.9049 ,   0.     ,
          0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
          0.     ,   0.     ,   0.     ,   0.     ],
       [  5.31441,   0.     ,   6.561  ,   0.     ,   0.     , -10.     ,
          0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
          0.     ,   0.     ,   0.     ,   0.     ],
       [  0.     ,   5.9049 ,   0.     ,   5.9049 ,   0.     ,   0.     ,
          7.29   ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
          0.     ,   0.     ,   0.     ,   0.     ],
       [  0.     ,   0.     ,   6.561  ,   0.     ,   0.     ,   0.     ,
          0.     , -10.     ,   0.     ,   0.     ,   0.     ,   0.     ,
          0.     ,   0.     ,   0.     ,   0.     ],
       [  5.31441,   0.     ,   0.     ,   0.     ,   0.     , -10.     ,
          0.     ,   0.     ,   6.561  ,   0.     ,   0.     ,   0.     ,
          0.     ,   0.     ,   0.     ,   0.   

In [16]:
def bestAction(state1):
    env = Env()
    env.current_state = numcod[state1]
    rew = []
    for ac in posact[state1]:
        obs,done,reward = env.step(ac)
        state2 = (obs[0]*4)+obs[1]
        rew.append(qmatrix[state1][state2])
        env.current_state = numcod[state1]
    action = posact[state1][np.argmax(rew)]
    return action

In [22]:
#play
state1 = 
done = False
env = Env()
count = 0
env.current_state = (0,1)
env.render()
while done != True:
    ran = rd.random()
    action = bestAction(state1)
    obs,done,reward = env.step(action)
    state2 = (obs[0]*4)+obs[1]
    qmatrix[state1][state2] = rmatrix[state1][state2] + y * max([qmatrix[state2][a] for a in list(range(16))])  
    state1 = state2
    env.render()
    count += 1
    if count>50:
        break


FSFF
FHFH
FFFH
HFFG

FFSF
FHFH
FFFH
HFFG

FFFF
FHSH
FFFH
HFFG

FFFF
FHFH
FFSH
HFFG

FFFF
FHFH
FFFH
HFSG

FFFF
FHFH
FFFH
HFFS



In [8]:
class Env:
    def __init__(self):
        self.env = np.array([[0,0,0,0],
                            [0,-1,0,-1],
                            [0,0,0,-1],
                            [-1,0,0,2]])
        self.current_state = (0,0)
        self.shape = self.env.shape
        self.done = False
        self.reward = 0

    def terminate_state(self):
        row,col = self.current_state
        if self.env[row][col] == 2:
            self.done = True
            self.reward = 10
        elif self.env[row][col] == -1:
            self.done = True
            self.reward = -10
        else:
            self.done = False
            self.reward = 0
    def reset(self):
        self.env = np.array([[0,0,0,0],
                            [0,-1,0,-1],
                            [0,0,0,-1],
                            [-1,0,0,2]])
        self.current_state = (0,0)
        self.shape = self.env.shape
        self.done = False
        self.reward = 0
        

    def step(self,action):
        row,col = self.current_state
        if action == 2:
            if col+1 < self.shape[1]:
                self.current_state = (row,col+1)
        if action == 0:
            if col-1 >= 0:
                self.current_state = (row,col-1)
        if action == 1:
            if row-1 >= 0:
                self.current_state = (row-1,col)
        if action == 3:
            if row+1 < self.shape[0]:
                self.current_state = (row+1,col)

        self.terminate_state()
        return self.current_state , self.done , self.reward


    def render(self):
        row,col = self.env.shape
        env_rend = ''
        for i in range(row):
            for j in range(col):
                state = self.env[i][j]
                if self.current_state == (i,j):
                    env_rend += 'S'
                elif state == 0:
                    env_rend += 'F'
                elif state == -1:
                    env_rend += 'H'
                else:
                    env_rend += 'G'
            env_rend += '\n'
        print(env_rend)

In [13]:
maze = Env()
maze.step(2)
maze.render()

FSFF
FHFH
FFFH
HFFG



In [14]:
maze.step(3)
maze.render()

FFFF
FSFH
FFFH
HFFG



In [15]:
maze.step(2)
maze.render()

FFFF
FHSH
FFFH
HFFG



In [11]:
maze.reset()
maze.render()

SFFF
FHFH
FFFH
HFFG



In [12]:
maze.step(2)

((0, 1), False, 0)